In [1]:
import cv2
import os
import random
import time
from datetime import datetime
import numpy as np
import matplotlib.pyplot as plt
import PIL.Image # For face recognition we will the the LBPH Face Recognizer
import pickle
from tkinter import *
from tkinter import ttk  
import tkinter as tk
import tkinter.messagebox
from PIL import Image
import win32com.client as win
#from sklearn.datasets import fetch_lfw_people

In [2]:
#detect + take pic
def new_user():
    image_path = 'Data/images/'
    face_cascade = cv2.CascadeClassifier('data/haarcascade/haarcascade_frontalface_default.xml')
    # Initializing the face detector and facial landmark predictor 
    #detector = dlib.get_frontal_face_detector()
    #predictor = dlib.shape_predictor("data/shape_predictor_68_face_landmarks.dat")

    cap = cv2.VideoCapture(0)
    cap.set(3, 1024) # set video widht
    cap.set(4, 720)
    name = input('enter user :')
    count = 0
    id_rand = random.sample(range(1,10000),1)
    myList = os.listdir(image_path)

    for line in myList:
        Type = line.split(".")
        split = os.path.splitext(line)
        if len(Type) > 2:
            y = Type[1]
        elif len(split) >= 2:
            y = split[1]
        if id_rand == y:
            id_rand = random.sample(range(1,10000),1)

    while cap.isOpened():
        ret, frame = cap.read()
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        new_path = image_path+str(name)+"."+ str(id_rand)      
        faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5)
        
        for (x, y, w, h) in faces: 
            #cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
            count+=1 
            if not os.path.exists(new_path):
                os.makedirs(new_path)
            cv2.imwrite(new_path+"/"+str(name)+"."+str(count)+ ".jpg", gray[y:y+h, x:x+w])
            cv2.rectangle(frame,(x,y),(x+w,y+h),(255,0,0),2)
            cv2.waitKey(100)
            if count == 15:
                cap.release()
                cv2.destroyAllWindows()
            elif cv2.waitKey(1) == 13: #13 is the Enter Key
                break

        cv2.imshow('cam',frame)
        cv2.destroyAllWindows()

In [3]:
def training(path):
    myList = os.listdir(path)
    for line in myList:
        Type = line.split(".")
        split = os.path.splitext(line)
        if len(Type) < 2:
            id_ = random.sample(range(10001,100000),1)
            src=path+line
            dst=path+Type[0]+'.'+str(id_)
            os.rename(src,dst)
        else:
            id = Type[1]
            name = Type[0]
            id = split[1]
            
    recognizer = cv2.face.LBPHFaceRecognizer_create() 
    faceCascade = cv2.CascadeClassifier("data/haarcascade/haarcascade_frontalface_alt.xml")
 
    baseDir = os.path.dirname(os.path.abspath("__file__"))
    #train the images under the images folder
    imageDir = os.path.join(baseDir, path)
    xTrain = []
    yLabels = []
    currentId = 1
    labelIds = {}
    
    for root, dirs, files in os.walk(imageDir):
        for file in files:
            if file.endswith("png") or file.endswith("jpg"):                
                label = os.path.basename(root)
                if not label in labelIds:
                    labelIds[label] = currentId
                    currentId += 1
                id_ = labelIds[label]
                
                path = os.path.join(root, file)
                pilImage = PIL.Image.open(path).convert("L")
                size = (160, 160)
                fin_img = pilImage.resize(size, Image.ANTIALIAS)
                imageArray = np.array(fin_img)
                faces = faceCascade.detectMultiScale(imageArray, scaleFactor=1.1, minNeighbors=5)
                
                for (x, y, w, h) in faces:                    
                    roi = imageArray[y:y+h, x:x+w]
                    xTrain.append(roi)
                    yLabels.append(id_)

    xTrain = np.asarray(xTrain)
    trainY = np.array(yLabels)
    try:
        print('\033[1m' + str(xTrain.shape + trainY.shape))
    except:
        print('\033[1m' + 'problem with shape')
        
    with open("data/pickles/labels_haar", "wb") as f:
        pickle.dump(labelIds, f)
        f.close()
    #recognizer.update(xTrain, trainY)
    recognizer.train(xTrain, trainY)
    recognizer.save("data/trainer/trainingdata_haar.yml")

In [4]:
#recognizer
def recognizer_face():
    with open('data/pickles/labels_haar', 'rb') as f:
        dicti = pickle.load(f)
        f.close()
    # Initialize and start realtime video capture
    cap = cv2.VideoCapture(0)
    cap.set(3, 800) # set video widht
    cap.set(4, 600)
    # Define min window size to be recognized as a face
    minW = 0.1*cap.get(3)
    minH = 0.1*cap.get(4)
    #class to detect objects in a video stream
    faceCascade = cv2.CascadeClassifier("data/haarcascade/haarcascade_frontalface_alt.xml")
    font = cv2.FONT_HERSHEY_SIMPLEX
    recognizer = cv2.face.LBPHFaceRecognizer_create()
    recognizer.read("data/trainer/trainingdata_haar.yml")

    #for frame in camera.capture_continuous(rawCapture, format="bgr", use_video_port=True):
    while True:
        ret,frame = cap.read()   
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        faces = faceCascade.detectMultiScale(gray, scaleFactor = 1.5, minNeighbors = 5)
        for (x, y, w, h) in faces:
            cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 185, 93), 2)
            roiGray = gray[y:y+h, x:x+w]
            
            id_, conf  = recognizer.predict(roiGray)
            for name, value in dicti.items():
                if value == id_:
                    #print(name)
                    Typ = name.split(".")
                    only_name = Typ[0]
                    last = ''
                    if only_name != last :
                        last = only_name                
            
            speak=win.Dispatch("SAPI.spVoice")
            speak.rate=-1
            color = (127,63,0)
            if conf >= 40:
                #cv2.putText(frame, only_name+'/'+str("{:.2f}".format(conf)), (x, y), font, 0.5,color,1,cv2.LINE_AA)
                cv2.putText(frame,  only_name+'/'+str("{:.2f}".format(conf)), (int(x+w+15), int(y-64)), font, 1, color, 2)
                #connect face and text
                cv2.line(frame,(x+w, y-64),(x+w-25, y-64),color,1)
                cv2.line(frame,(int(x+w/2),y),(x+w-25,y-64),color,1)
                
            else:
                text = 'Unkown'
                #cv2.putText(frame, text+'/'+str("{:.2f}".format(conf)), (x, y), font, 0.5,(255,255,255),1,cv2.LINE_AA)
                cv2.putText(frame,  text, (int(x+w+15), int(y-64)), font, 1, color, 2)
                #connect face and text
                cv2.line(frame,(x+w, y-64),(x+w-25, y-64),color,1)
                cv2.line(frame,(int(x+w/2),y),(x+w-25,y-64),color,1)
                speak.speak(name)

            with open('data/attendance/Attendance_haar.csv','r+') as f:
                myDataList = f.readlines()
                nameList = []
                for line in myDataList:
                    entry = line.split(',')
                    nameList.append(entry[0])
                if only_name not in nameList:
                    now = datetime.now()
                    dtString = now.strftime('%H:%M:%S')
                    f.writelines(f'\n{only_name},{dtString}')
                    speak.speak(name)
        cv2.imshow('frame', frame)
        key = cv2.waitKey(1)
        if key == 13:
            break

    cap.release()
    cv2.destroyAllWindows()  

In [7]:
path ="data/images/"
def F_popup():
    pop_up = tk.Tk()
    pop_up.wm_title('Facial Recognition')
    pop_up.geometry("700x100")
    b1 = ttk.Button(pop_up, text='Train model', command=lambda:[training(path)])
    b1.pack(side='left',pady=10,fill='both', expand=True)
    b2 = ttk.Button(pop_up, text='Facial recognition', command=recognizer_face)
    b2.pack(side='left',pady=10,fill='both', expand=True)
    b3 = ttk.Button(pop_up, text='Close', command=pop_up.destroy)
    b3.pack(side='right', pady=10,fill='both', expand=True)
    pop_up.mainloop()
    
pop_up = tk.Tk()
pop_up.wm_title('Facial Recognition')
pop_up.geometry("500x100")
b1 = ttk.Button(pop_up, text='New users', command=new_user)
b1.pack(side='left',pady=10,fill='both', expand=True)
b2 = ttk.Button(pop_up, text='continue', command = lambda:[pop_up.destroy(),F_popup()])
b2.pack(side='right', pady=10,fill='both', expand=True)
pop_up.mainloop()

enter user : adem


(36, 36)


In [6]:
#with open('data/Attendance_haar.csv','w+') as f:
    #pass  
    